In [1]:
import tensorflow as tf

In [2]:
import os

In [3]:
from  PIL import Image

In [4]:
sess = tf.InteractiveSession()

In [5]:
import glob

In [6]:
image_filenames = glob.glob("./Images/n*/*.jpg")

In [7]:
image_filenames[0:2]

['./Images/n02097658-silky_terrier/n02097658_26.jpg',
 './Images/n02097658-silky_terrier/n02097658_4869.jpg']

In [8]:
from itertools import groupby
from collections import defaultdict

In [9]:
training_dataset = defaultdict(list)
testing_dataset = defaultdict(list)

In [10]:
image_filename_with_breed = map(lambda filename: (filename.split("/")[2], filename),
                                image_filenames)

In [11]:
for dog_breed, breed_images in groupby(image_filename_with_breed, lambda x: x[0]):
    
    for i, breed_image in enumerate(breed_images):
        if i % 5 == 0:
            testing_dataset[dog_breed].append(breed_image[1])
        else:
            training_dataset[dog_breed].append(breed_image[1])
    
    breed_training_count = len(training_dataset[dog_breed])
    breed_testing_count = len(testing_dataset[dog_breed])
    
    breed_training_count_float = float(breed_training_count)
    breed_testing_count_float = float(breed_testing_count)
    
    assert round(breed_testing_count_float / (breed_training_count_float + breed_testing_count_float), 2) > 0.18, "Not enough testing images."
    
    print "training dataset testing dataset  END-------------------------------------------------------------------"

training dataset testing dataset  END-------------------------------------------------------------------
training dataset testing dataset  END-------------------------------------------------------------------
training dataset testing dataset  END-------------------------------------------------------------------
training dataset testing dataset  END-------------------------------------------------------------------
training dataset testing dataset  END-------------------------------------------------------------------
training dataset testing dataset  END-------------------------------------------------------------------
training dataset testing dataset  END-------------------------------------------------------------------
training dataset testing dataset  END-------------------------------------------------------------------
training dataset testing dataset  END-------------------------------------------------------------------
training dataset testing dataset  END------------------

In [ ]:
def write_records_file(dataset, record_location):
    
    if not os.path.exists(record_location):
        print(" %s not exist, creating..."  % (record_location))
        os.makedirs(record_location)
        
    writer = None
    
    current_index = 0
    
    for breed, images_filenames in dataset.items():
        for image_filename in images_filenames:
            if current_index % 100 == 0:
                if writer:
                    writer.close()

                record_filename = "{record_location}-{current_index}.tfrecords".format(
                    record_location=record_location,
                    current_index=current_index)

                writer = tf.python_io.TFRecordWriter(record_filename)
                print record_filename + "------------------------------------------------------------------"
            current_index += 1

            image_file = tf.read_file(image_filename)

            # In ImageNet dogs, there are a few images which TensorFlow doesn't recognize as JPEGs. This
            # try/catch will ignore those images.
            try:
                image = tf.image.decode_jpeg(image_file)
            except:
                print(image_filename)
                continue

            # Converting to grayscale saves processing and memory but isn't required.
            grayscale_image = tf.image.rgb_to_grayscale(image)
            resized_image = tf.image.resize_images(grayscale_image, [250, 151])

            # tf.cast is used here because the resized images are floats but haven't been converted into
            # image floats where an RGB value is between [0,1).
            image_bytes = sess.run(tf.cast(resized_image, tf.uint8)).tobytes()

            
            #image = Image.open(image_filename)
            #image = image.resize((IMAGE_WIDTH, IMAGE_HEIGHT))
            #image_bytes = image.tobytes()  # 将图片转成二进制
            # Instead of using the label as a string, it'd be more efficient to turn it into either an
            # integer index or a one-hot encoded rank one tensor.
            # https://en.wikipedia.org/wiki/One-hot
            image_label = breed.encode("utf-8")

            example = tf.train.Example(features=tf.train.Features(feature={
                'label': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_label])),
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes]))
            }))

            writer.write(example.SerializeToString())
    writer.close()

#write_records_file(testing_dataset, "./output/testing-images/testing-image")
write_records_file(training_dataset, "./output/training-images/training-image")
print "writing records_file END--------------------------------------------------------------------------"

./output/training-images/training-image-0.tfrecords------------------------------------------------------------------
./output/training-images/training-image-100.tfrecords------------------------------------------------------------------
./output/training-images/training-image-200.tfrecords------------------------------------------------------------------
./output/training-images/training-image-300.tfrecords------------------------------------------------------------------
./output/training-images/training-image-400.tfrecords------------------------------------------------------------------
./output/training-images/training-image-500.tfrecords------------------------------------------------------------------
./output/training-images/training-image-600.tfrecords------------------------------------------------------------------
./output/training-images/training-image-700.tfrecords------------------------------------------------------------------
./output/training-images/training-image-80

In [ ]:
filename_queue = tf.train.string_input_producer(
    tf.train.match_filenames_once("./output/training-images/*.tfrecords"))
reader = tf.TFRecordReader()
_, serialized = reader.read(filename_queue)

features = tf.parse_single_example(
    serialized,
    features={
        'label': tf.FixedLenFeature([], tf.string),
        'image': tf.FixedLenFeature([], tf.string),
    })

record_image = tf.decode_raw(features['image'], tf.uint8)

# Changing the image into this shape helps train and visualize the output by converting it to
# be organized like an image.
image = tf.reshape(record_image, [250, 151, 1])

label = tf.cast(features['label'], tf.string)

min_after_dequeue = 10
batch_size = 3
capacity = min_after_dequeue + 3 * batch_size
image_batch, label_batch = tf.train.shuffle_batch(
    [image, label], batch_size=batch_size, capacity=capacity, min_after_dequeue=min_after_dequeue)
print " load image form TFRcord  END---------------------------------------------------"

In [ ]:
# Converting the images to a float of [0,1) to match the expected input to convolution2d
float_image_batch = tf.image.convert_image_dtype(image_batch, tf.float32)

conv2d_layer_one = tf.contrib.layers.convolution2d(
    float_image_batch,
    num_output_channels=32,     # The number of filters to generate
    kernel_size=(5,5),          # It's only the filter height and width.
    activation_fn=tf.nn.relu,
    weight_init=tf.random_normal,
    stride=(2, 2),
    trainable=True)
pool_layer_one = tf.nn.max_pool(conv2d_layer_one,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding='SAME')

# Note, the first and last dimension of the convolution output hasn't changed but the
# middle two dimensions have.
conv2d_layer_one.get_shape(), pool_layer_one.get_shape()
print "conv2d_layer_1 pool_layer_1  END---------------------------------------------------"